---

# 🎯 Home Assignment: AI Agent for Breast Cancer Support

## Goal

Build an AI agent that interacts with a woman recently diagnosed with breast cancer, asking **personalized and relevant questions** based on her profile. This simulates the kind of **human-centered, empathetic AI assistant**

---

## ✅ Part 1: Build an Empathetic AI Agent

Your agent should:

1. Ask the user for their **first name**
2. Ask for their **age**
3. Ask about the **type of breast cancer diagnosis**
4. Ask for the **treatment they are currently receiving** (if any)
5. Ask if they have a **support system** (e.g., family, friends, community)

### 💡 Important Nuance

The user may not answer on the first try. Your agent must demonstrate:

* **Polite persistence**
* **Empathy and emotional sensitivity**
* The ability to handle **evasive, partial, or delayed responses**

---

## ⭐️ Part 2: Bonus Challenge – Dynamic Dialogue Generation

Use a free-form condition summary provided by the user to generate relevant, emotionally sensitive questions.

### Sample Input

```python
condition_profile = {
    "age": 45,
    "diagnosis": "Triple-negative breast cancer",
    "stage": "Stage IIb",
    "treatment": "AC-T chemotherapy just started",
    "next_steps": "Genetic testing and breast MRI",
}
```

### Your agent should:

* Parse this profile intelligently
* Generate **5 dynamic, emotionally sensitive questions**
* Run the **Q\&A dialogue** with the user
* Account for **hesitancy, deflection, or partial answers**

#### ✨ Example Questions:

* “Would you like to share how you’re feeling about starting chemotherapy?”
* “Have you been offered genetic counseling yet, or is that still ahead?”
* “Do you have access to information about AC-T and how to manage side effects?”
* “Are you feeling supported emotionally right now?”
* “Is there anything you’re anxious about in the next few weeks?”

---

## ⚙️ Tools & Libraries

You may use any of the following (or others):

* OpenAI / Anthropic APIs
* LangChain / Haystack
* spaCy / NLTK / Hugging Face Transformers
* Your own dialogue logic or frameworks

---

## 📦 Submission Guidelines

Submit your solution via:

* ✅ A Google Colab notebook, **or**
* ✅ A GitHub repo, **or**
* ✅ A short explainer video + code (if you'd like to demo interactivity)

Your code should:

* Be **readable and well-organized**
* Handle **edge cases** (e.g., missing input, unclear answers)
* Demonstrate **empathy** in both wording and flow
* Include **brief comments** explaining your design choices

---

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00


In [ ]:
#the notebook use openapi key with name "open_ai"
from google.colab import userdata
OPENAI_API_KEY = userdata.get('open_ai')

### First Solution :
prompt + json without structured output

In [ ]:
import openai
import gradio as gr
from pydantic import BaseModel, Field, ValidationError
from typing import Optional
import os

openai.api_key = OPENAI_API_KEY

# Define user profile schema
class UserProfile(BaseModel):
    name: Optional[str] = Field(None, description="First name of the user")
    age: Optional[int] = Field(None, ge=18, le=120, description="Age of the user")
    diagnosis: Optional[str] = Field(None, description="Type of breast cancer")
    treatment: Optional[str] = Field(None, description="Current treatment")
    support: Optional[str] = Field(None, description="Support system")

# Initialize state
conversation = []
user_profile = UserProfile()

# System prompt that guides GPT behavior
SYSTEM_PROMPT = {
    "role": "system",
    "content": (
        "You are an AI support companion for women recently diagnosed with breast cancer. "
        "Your primary role is to offer empathetic listening, emotional validation, and compassionate support. "
        "Woven into your support, you are also tasked with gently gathering specific information needed to potentially tailor future support and resources.\n\n"
        "The key information you need to gather, one piece at a time, includes:\n"
        "- **First name**\n"
        "- **Age** (Ensure it's within a reasonable adult range)\n"
        "- **Type of breast cancer diagnosis**\n"
        "- **Current treatment plan** (if any)\n"
        "- **Support system** (e.g., family, friends, community, support groups)\n\n"
        "**Interaction Guidelines:**\n"
        "1.  **Prioritize Empathy and Support:** Always respond with compassion. Use reflective listening ('It sounds like you're feeling...') and validate their emotions before asking a question or steering the conversation.\n"
        "2.  **Ask ONE Question at a Time:** Introduce profile questions naturally and ask for only one piece of information per turn. Do not ask multiple questions in a single response.\n"
        "3.  **Gentle Persistence:** If the user doesn't provide a piece of information or changes the subject, find a natural, empathetic way to gently guide the conversation back to the remaining questions *when appropriate* and after addressing their immediate needs.\n"
        "4.  **Handle Digressions:** If the user wants to talk about something else or expresses strong feelings, allow them to do so. Listen actively and provide support. Only attempt to ask a profile question or steer back *after* you have validated their feelings and addressed their immediate emotional needs.\n"
        "5.  **Clarification:** If an answer is ambiguous or potentially unclear (e.g., 'I'm having chemo' - you might need to gently ask about the *type* if relevant to the fields, or confirm the age if it seems inconsistent), ask a clarifying question before updating the JSON.\n"
        "6.  **Never Guess or Fabricate:** Only include information in the JSON that the user has explicitly and clearly provided. If information is missing, leave the corresponding field as `null`.\n"
        "7.  **Goal: Complete Profile:** Continue the conversation and gentle inquiry until you have successfully gathered all the required information for the profile fields or the user indicates they cannot or will not provide a specific piece of information.\n\n"
        "**Output Format:**\n"
        "After each response to the user, you *must* include the complete current state of the gathered profile information in a JSON block formatted exactly like this:\n"
        "```json\n{\"name\": null, \"age\": null, \"diagnosis\": null, \"treatment\": null, \"support\": null}\n```\n"
        "Update the JSON only with information the user explicitly provides and you confirm. Fields for which you have no confirmed information must remain `null`.\n\n"
        "**Initial Message:** Start the conversation with a warm, welcoming, and explanatory message outlining your purpose in a supportive way (e.g., 'Hello, I'm here to listen and support you. To help me understand how I can best assist you, I might gently ask a few questions about your journey...')."
    )
}

conversation.append(SYSTEM_PROMPT)

# Function to extract JSON block from GPT output
def extract_json(text):
    import json, re
    match = re.search(r"```json\n(.*?)\n```", text, re.DOTALL)
    if match:
        try:
            return json.loads(match.group(1))
        except json.JSONDecodeError:
            return None
    return None

# Function to handle chatbot interaction
def chat(user_input):
    global user_profile, conversation

    conversation.append({"role": "user", "content": user_input})

    response = openai.chat.completions.create(
        model="gpt-4",
        messages=conversation,
        temperature=0.7,
    )

    assistant_message = response.choices[0].message.content
    conversation.append({"role": "assistant", "content": assistant_message})

    # Try to extract and validate JSON
    extracted = extract_json(assistant_message)
    if extracted:
        try:
            user_profile = UserProfile(**{**user_profile.model_dump(), **extracted})
        except ValidationError as e:
            print("Validation error:", e)

    return assistant_message

# Gradio interface
def respond(message, history):
    reply = chat(message)
    return reply

with gr.Blocks() as demo:
    gr.Markdown("""# 🤖 Empathetic Breast Cancer Support Bot
Please talk to the bot. It's here to gently gather a few details and support you emotionally.
""")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Type here and press Enter...", label="Your Message")

    def handle_input(message, history):
        reply = respond(message, history)
        history.append((message, reply))
        return "", history

    msg.submit(handle_input, [msg, chatbot], [msg, chatbot])

demo.launch(debug=True)

<ipython-input-6-b5d5bf30d5c8>:97: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://22636b3883f43c4103.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://22636b3883f43c4103.gradio.live


## Second solution:
similar approch just with structured output,
forcing the llm to return strict json

In [ ]:
import gradio as gr
from pydantic import BaseModel
from typing import Optional
from openai import OpenAI
from google.colab import userdata  # or os.environ for local

# Init OpenAI
client = OpenAI(api_key=userdata.get("open_ai"))

# Schema with required chat response
class UserProfile(BaseModel):
    name: Optional[str] = None
    age: Optional[int] = None
    diagnosis: Optional[str] = None
    treatment: Optional[str] = None
    support: Optional[str] = None
    chat_response: str  # required for natural flow

# Initialize state
conversation = []
user_profile = UserProfile(chat_response="")

# System prompt with behavioral instructions
SYSTEM_PROMPT = {
    "role": "system",
    "content": (
        "You are an empathetic AI support agent. Your **primary goal** is to collect the following five fields from the user:\n"
        "- name (first name only)\n"
        "- age (must be a number)\n"
        "- diagnosis (type of breast cancer)\n"
        "- treatment (if any)\n"
        "- support system (family, friends, community)\n\n"
        "You must gather this information **step by step**.\n"
        "Start by asking for their name. Then move to age. Then diagnosis. Then treatment. Then support system.\n"
        "At each step:\n"
        "- Be warm and emotionally sensitive.\n"
        "- If the user avoids the question, gently and politely ask again using different words.\n"
        "- Acknowledge feelings, but always try to gather the missing field.\n\n"
        "Once a field is collected, do not ask it again.\n"
        "ALWAYS respond in this JSON format:\n"
        "{\n"
        "  \"name\": null or string,\n"
        "  \"age\": null or number,\n"
        "  \"diagnosis\": null or string,\n"
        "  \"treatment\": null or string,\n"
        "  \"support\": null or string,\n"
        "  \"chat_response\": \"string with your human message\"\n"
        "}\n\n"
        "Example:\n"
        "{\n"
        "  \"name\": \"Sarah\",\n"
        "  \"age\": null,\n"
        "  \"diagnosis\": null,\n"
        "  \"treatment\": null,\n"
        "  \"support\": null,\n"
        "  \"chat_response\": \"Thank you, Sarah. May I ask how old you are?\"\n"
        "}"
    )
}


conversation.append(SYSTEM_PROMPT)

# Chat logic
def chat(user_input):
    global user_profile, conversation

    conversation.append({"role": "user", "content": user_input})

    response = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=conversation,
        response_format=UserProfile,
        temperature=0.7,
    )

    parsed = response.choices[0].message.parsed
    merged = {
        **user_profile.model_dump(),
        **{k: v for k, v in parsed.model_dump().items() if v is not None}
    }
    user_profile = UserProfile(**merged)

    assistant_message = user_profile.chat_response
    conversation.append({"role": "assistant", "content": assistant_message})

    # When all fields are complete (excluding chat_response), end flow
    if all(getattr(user_profile, f) is not None for f in UserProfile.model_fields if f != "chat_response"):
        final = (
            f"Thank you so much, {user_profile.name}. You've been very brave sharing all this.\n"
            "If there's anything else you’d like to talk about, I’m always here for you. 💖"
        )
        conversation.append({"role": "assistant", "content": final})
        return final

    return assistant_message

# Gradio UI
def respond(msg, history):
    reply = chat(msg)
    history.append((msg, reply))
    return "", history

with gr.Blocks() as demo:
    gr.Markdown("## 🤖 Empathetic Breast Cancer Support Agent\n_Gently gathering your information with care._")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Share what you're comfortable with...", label="You")

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(debug=True)


<ipython-input-40-e1f90229cdfb>:106: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4974a7c0a7369e2a91.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


##Bonus Challenge (Dynamic Dialogue Generation)

In [ ]:
# Import necessary libraries
import openai
import gradio as gr
import json
import os
from typing import List, Dict, Any, Optional
from pydantic import BaseModel
from enum import Enum

from google.colab import userdata
openai.api_key = userdata.get("open_ai")


# 1. Generate personalized, empathetic questions from a patient profile using LLM
def generate_questions_from_profile(profile_dict: Dict[str, Any]) -> List[str]:
    """
    Generates a list of 5 empathetic questions tailored to a user profile using LLM.
    Returns a list of question strings or a fallback list if generation fails.
    """
    prompt = f"""
You are an AI assistant specializing in generating empathetic and relevant questions for a support agent interacting with a breast cancer patient.
Your task is to create 5 DISTINCT, emotionally sensitive questions based on the provided patient profile. These questions will be used sequentially by the support agent.

Each question must:
- Be warm, human, and respectful.
- Be specifically tailored to the details in THIS user's profile (diagnosis, treatment, next steps, age, etc.).
- Focus on feelings, experiences, concerns, or practical support needs, NOT just factual medical details.
- Be phrased gently and be open-ended to encourage sharing.
- Be suitable to be asked one at a time in a supportive conversation.

Output ONLY a JSON list of exactly 5 strings (the questions). Do not include any other text before or after the JSON.

Patient Profile:
{json.dumps(profile_dict, indent=2)}
"""

    fallback_questions = [
        "How are you feeling today?",
        "Do you have support around you?",
        "What's been on your mind the most?",
        "Is there anything specific you're finding challenging?",
        "What's one small thing that brought you comfort recently?"
    ]

    # Schema with required chat response list of 5 strings
    class UserQuestions(BaseModel):
        questions: List[str]


    try:
        # Use the standard create method, asking for JSON output
        response = openai.beta.chat.completions.parse(
            model="gpt-4o",
            messages=[{"role": "system", "content": prompt}],
            response_format=UserQuestions,
            temperature=0.7,
        )

        parsed = response.choices[0].message.parsed
        questions = parsed.questions


        if isinstance(questions, list) and len(questions) == 5 and all(isinstance(q, str) for q in questions):
            print("--- Generated Questions ---")
            for i, q in enumerate(questions):
                print(f"{i+1}. {q}")
            print("-------------------------")
            return questions
        else:
            print(f"Error: LLM returned incorrect format for questions. Expected list of 5 strings, got: {text}")
            return fallback_questions
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from LLM response (questions): {e}\nResponse text: {text}")
        return fallback_questions
    except Exception as e:
        print(f"An API error occurred during question generation: {e}")
        return fallback_questions

# --- Structured Output Definition for Dialogue Turn ---

class DialogueAction(str, Enum):
    """Enum to signal the next state action in the dialogue."""
    MOVE_NEXT = "move_next"         # User addressed current question, move to next
    STAY_CURRENT = "stay_current"   # User did not address current question, stay on it (rephrase/wait)
    CONCLUDE = "conclude"           # Dialogue finished (all questions done or conversation ending)

class DialogueResponse(BaseModel):
    """Pydantic model for the structured dialogue turn output."""
    ai_message: str                 # The natural language message for the user
    current_question_addressed: bool # True if the user's last message sufficiently addressed the current question being focused on
    next_action: DialogueAction     # The recommended next action for the dialogue state
    model_config = {
        "json_schema_extra": {
        }
    }


# --- State Management for the Dialogue ---
class ChatState:
    """Manages the state of the Q&A dialogue."""
    def __init__(self):
        self.questions: List[str] = [] # The list of generated questions
        self.current_q_index: int = 0 # Index of the question currently being focused on
        self.dialogue_history: List[Dict[str, str]] = [] # Stores the full conversation history for the dialogue LLM

# Initialize state globally (common pattern for Gradio state)
state = ChatState()

# 2. Main Dialogue Handler (using client.beta.chat.completions.parse with Pydantic)
def main_dialogue_turn_structured(user_input: str) -> DialogueResponse:
    """
    Processes user input, updates dialogue history, and generates the AI's next response
    and state update based on the predefined questions and empathetic persona,
    using structured output with Pydantic parse method.

    Args:
        user_input: The user's message for the current turn.

    Returns:
        A DialogueResponse Pydantic object containing the AI's message and state signals,
        or a fallback DialogueResponse in case of API/parsing errors.
    """
    # Add user message to internal dialogue history
    state.dialogue_history.append({"role": "user", "content": user_input})

    # Get context for the prompt
    current_question = state.questions[state.current_q_index] if state.current_q_index < len(state.questions) else None
    next_question_index = state.current_q_index + 1

    # --- Prepare the prompt for the dialogue LLM using structured output instructions ---
    dialogue_prompt_content = f"""
You are an empathetic support agent assisting a woman recently diagnosed with breast cancer.
Your primary role is to provide genuine emotional support, listen actively, and validate feelings.
Your secondary goal is to gently guide the conversation to get responses to a list of specific questions ({len(state.questions)} total) to understand the user better and tailor support.

You MUST output your response as a JSON object strictly adhering to the Pydantic schema for `DialogueResponse`. The schema is:
{json.dumps(DialogueResponse.model_json_schema(), indent=2)}

Based on the conversation history, the user's last message, and your list of questions, populate the JSON fields:
- `ai_message`: Your complete, natural language response (string). This should be empathetic validation followed by the next conversational turn.
- `current_question_addressed`: Boolean indicating if the user's *last message* gave a sufficient response or acknowledgement related to the question you were focused on (Question #{state.current_q_index + 1}).
- `next_action`: String enum (`"move_next"`, `"stay_current"`, `"conclude"`).
    - `"move_next"`: If `current_question_addressed` is true AND there are more questions left. Your `ai_message` should gently lead to Question #{next_question_index + 1}.
    - `"stay_current"`: If `current_question_addressed` is false. Your `ai_message` should rephrase or offer support around the current question, not ask the next one.
    - `"conclude"`: If `current_question_addressed` is true AND this was the last question. Your `ai_message` should be a concluding supportive remark.

Here is the list of questions you are guiding the user through:

<Questions>
{chr(10).join([f"{i+1}. {q}" for i, q in enumerate(state.questions)])}
</Questions>

{f"Your current focus is on Question #{state.current_q_index + 1}: '{current_question}'" if current_question else "All questions have been covered. Provide ongoing support if needed."}

Conversation History (Most Recent Last):
{chr(10).join([f"{msg['role']}: {msg['content']}" for msg in state.dialogue_history])}

Your output MUST be ONLY the JSON object. Do not include any extra text before or after.
"""
    # --- Perform the LLM call using the beta parse method ---
    try:
        # Using the beta parse method with the Pydantic schema
        parsed_completion = openai.beta.chat.completions.parse(
            model="gpt-4o", # Must use a model that supports json_schema response_format
            messages=[
                {"role": "system", "content": dialogue_prompt_content}
            ],
            response_format=DialogueResponse, # Pass the Pydantic model directly
            temperature=0.7, # Allows for empathetic phrasing variation in ai_message
            max_tokens=800 # Increased max tokens slightly to ensure full JSON and message fit
        )

        parsed_response = parsed_completion.choices[0].message.parsed

        # Add AI message from the parsed response to internal history
        state.dialogue_history.append({"role": "assistant", "content": parsed_response.ai_message})

        return parsed_response # Return the actual Pydantic model instance

    except Exception as e:
        print(f"An API error occurred during dialogue turn or parsing: {e}")
        # --- Fallback structured response in case of error ---
        fallback_message = "I'm sorry, I'm having trouble responding right now due to a technical issue. Please feel free to share whatever is on your mind, or we can try returning to the questions later."
        # Add the fallback message to history
        state.dialogue_history.append({"role": "assistant", "content": fallback_message})
        # Return a structured response object indicating failure and staying on current question
        return DialogueResponse(
            ai_message=fallback_message,
            current_question_addressed=False, # Assume current question was not addressed if error
            next_action=DialogueAction.STAY_CURRENT # Try to stay on the current question/state
        )

# 3. Gradio Interface Handler
def respond(user_text: str, history: List[List[str]], status_text_value: str):
    """
    Handles user input during the dialogue, calls the structured dialogue handler,
    updates state based on structured output, and updates Gradio history and status.
    """
    # --- Input Validation ---
    if not user_text:
        # If empty input, just return current state - history unchanged, input text unchanged, status unchanged, interactive unchanged
        # We need to return values for outputs=[user_input, chatbox, status_text]
        # The input textbox should remain interactive if we are in the respond function
        return "", history, status_text_value # Return current values

    # --- Call Structured Dialogue Logic ---
    # Pass the user input to the structured dialogue handler
    parsed_response = main_dialogue_turn_structured(user_text)

    # --- Update State Based on Structured Output ---
    # Use the 'next_action' signal from the LLM to update the question index
    if parsed_response.next_action == DialogueAction.MOVE_NEXT:
        state.current_q_index += 1
    elif parsed_response.next_action == DialogueAction.CONCLUDE:
        state.current_q_index = len(state.questions) # Mark as finished by setting index past the end


    # --- Determine New Status Message ---
    if state.current_q_index >= len(state.questions):
        # All questions covered
        new_status_value = "Structured questions covered. You can continue chatting for general support."
    else:
        # Still questions left
        new_status_value = f"Focusing on question {state.current_q_index + 1}/{len(state.questions)}"

    # --- Update Gradio History ---
    # Append user message first, then the AI's response from the structured output
    # Note: history comes in as a copy in Gradio 4.0, modify and return it
    history.append([user_text, None]) # Append user input with placeholder for AI response
    history[-1][1] = parsed_response.ai_message # Update the AI's response in the last history entry


    # --- Determine Interactive State ---
    # The input should remain interactive unless we specifically want to disable it later.
    # For this flow, it stays interactive for potential general chat after questions.
    user_input_update = gr.update(interactive=True, value="") # Clear input box value, ensure interactive is True


    # --- Return Outputs ---
    # Return values matching the outputs list: [user_input, chatbox, status_text]
    return user_input_update, history, new_status_value


# 4. Gradio App Definition
with gr.Blocks() as demo:
    gr.Markdown("""
    # 🤖 Personalized Empathetic Support Agent (Dynamic Q&A with Structured Output)
    This demo dynamically generates questions based on a profile and guides you through them empathetically using structured AI responses.
    """)

    # Chatbox display
    chatbox = gr.Chatbot(label="Conversation")

    # User input textbox (initially disabled until questions are generated)
    user_input = gr.Textbox(label="Your Message", placeholder="Type your message and press Enter", interactive=False)

    # Status indicator (informative text like "Generating questions...", "Focusing on question X/Y", "Questions covered")
    status_text = gr.Textbox(label="Status", interactive=False)

    # --- Event Handlers ---

    def start_chat_process():
        """
        Initializes the chat flow upon demo load.
        Generates questions, sets initial state, prepares the first message,
        and enables the user input box.
        Returns values to update the output components.
        """
        # Define the profile to generate questions from
        profile = {
            "age": 45,
            "diagnosis": "Triple-negative breast cancer",
            "stage": "Stage IIb",
            "treatment": "AC-T chemotherapy just started",
            "next_steps": "Genetic testing and breast MRI",
            "note": "Patient is likely feeling overwhelmed and anxious about starting chemo and upcoming tests." # Added a note for more empathetic tailoring
        }

        # --- Initialization ---
        # Update status text to indicate processing
        status_text_update_value = "Generating personalized questions..."
        # Reset state for a new session
        state.__init__()

        # --- Generate Questions ---
        generated_questions = generate_questions_from_profile(profile)
        state.questions = generated_questions
        state.current_q_index = 0 # Start with the first question being index 0 (list is 0-indexed)

        # --- Prepare Initial Output ---
        if not state.questions or state.questions[0].startswith("I'm sorry"):
             # Question generation failed or returned fallback list starting with error message
             initial_message_value = state.questions[0] # Use the fallback message itself
             # Clean up state if generation failed
             state.questions = []
             state.current_q_index = 0 # Ensure index is 0 if no questions

             final_status_value = "Failed to generate personalized questions. You can chat for general support, but the structured Q&A flow is not active."
             # Enable user input so they can chat generally
             user_input_update = gr.update(interactive=True, value="")
             # Start chat history with the AI's initial (error) message
             initial_history_value = [(None, initial_message_value)]
             # Add to internal dialogue history
             state.dialogue_history.append({"role": "assistant", "content": initial_message_value})

        else:
             # Questions generated successfully
             # Set an initial AI welcome message. The first question will be asked by the AI
             # in response to the user's *first* input via main_dialogue_turn_structured.
             initial_message_value = "Hello, I'm here to listen and support you. To help me understand how I can best assist you, I might gently ask a few questions about your journey."
             # Add to internal dialogue history
             state.dialogue_history.append({"role": "assistant", "content": initial_message_value})

             final_status_value = f"Questions generated ({len(state.questions)}). We can now begin. Focusing on question {state.current_q_index + 1}/{len(state.questions)}. Please share how you're feeling or respond to the first question when you're ready."
             # Enable user input and clear any default value
             user_input_update = gr.update(interactive=True, value="")
             # Start chat history with the AI's welcome message
             initial_history_value = [(None, initial_message_value)]

        # --- Return Outputs for demo.load ---
        # The order of returns must match the order of components in the 'outputs' list below.
        # outputs=[chatbox, user_input, status_text]
        return initial_history_value, user_input_update, final_status_value


    # --- Gradio Event Binding ---
    # Initial load event: Triggers the start_chat_process function
    # This function updates the chatbox, enables/clears user_input, and updates status_text.
    demo.load(
        start_chat_process,
        inputs=None, # No inputs needed for initial load
        outputs=[chatbox, user_input, status_text] # Components to update
    )

    # User input submission event: Triggers the respond function
    # The respond function updates the chat history, clears user_input, and updates status_text.
    user_input.submit(
        respond,
        inputs=[user_input, chatbox, status_text], # Inputs needed: the text, the current history, the current status value
        outputs=[user_input, chatbox, status_text] # Components to update
    )


# Launch the Gradio demo
demo.launch(debug=True)

<ipython-input-13-91b290e8dffd>:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbox = gr.Chatbot(label="Conversation")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://53165312e9c951d03c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


--- Generated Questions ---
1. How are you feeling about starting your chemotherapy treatment, and is there anything specific that's been on your mind?
2. What concerns or thoughts do you have about the upcoming genetic testing and MRI, and how can I support you through this process?
3. With everything going on, how are you managing to find moments of calm or support in your daily life?
4. Are there any particular aspects of your treatment or condition that you find most challenging, and how can we help ease those burdens?
5. How do you feel about discussing your diagnosis and treatment with friends or family, and do you have the support you need from them?
-------------------------
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://53165312e9c951d03c.gradio.live
